<a href="https://colab.research.google.com/github/gitconnoisseur/toxicity-model/blob/master/ecfp_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyTDC
!pip install deepchem
!pip install autokeras
!pip install --upgrade numpy

In [3]:
#import autogluon as ag
from tdc.single_pred import Tox
import deepchem
import xgboost

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [4]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    # all benchmark names in a benchmark group are stored in group.dataset_names
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
    
        # --------------------------------------------- # 
        #  Train your model using train, valid, test    #
        #  Save test prediction in y_pred_test variable #
        # --------------------------------------------- #
    

    featurizer = deepchem.deepchem.feat.WeaveFeaturizer()
    f_train = featurizer.featurize(train.iloc[:,1].to_list())
    f_valid = featurizer.featurize(valid.iloc[:,1].to_list())

    train_data = deepchem.deepchem.data.NumpyDataset(X=f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    valid_data = deepchem.deepchem.data.NumpyDataset(X=f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    reg = deepchem.deepchem.models.WeaveModel(
        n_tasks=1,        
        mode="regression",)
    callback = deepchem.deepchem.models.ValidationCallback(valid_data, 1000, metric)
    reg.fit(train_data, nb_epoch=100, callbacks=callback)

    testFeatures = np.array(featurizer.featurize(test.iloc[:,1].to_list()))

    y_pred_test = reg.predict_on_batch(X=testFeatures)

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:01<00:00, 3802.89it/s]


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

In [ ]:
def model_builder(**model_params):
  #dropout = model_params['dropout']

  #min_samples_split = model_params['min_samples_split']

  #gcn_model = deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="regression", dropout=dropout)

  return deepchem.deepchem.models.GraphConvModel(n_tasks=1, mode="regression", graph_conv_layers=[128, 128, 128])

#optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

params_dict = {"dropout": .01, "dense_layer_size": 200}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_data, valid_data, metric, search_range=10)